In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
GFP = pd.read_csv("201119_platereader/GFP.csv", index_col=0)
del GFP["Temp fluor:485,515"]

mScarlet = pd.read_csv("201119_platereader/mScarlet.csv", index_col=0)
del mScarlet["Temp fluor:575,615"]

OD600 = pd.read_csv("201119_platereader/OD600.csv", index_col=0)
del OD600["Temp OD:600"]


for df in [GFP, mScarlet, OD600]:
    df.index = pd.to_timedelta(df.index)

df = pd.concat({"GFP": GFP, "mScarlet": mScarlet, "OD": OD600}, axis=1)

df = df.swaplevel(axis=1)
df.sort_index(axis=1, inplace=True)
df.interpolate(inplace=True)
idx = pd.IndexSlice

In [ ]:
Nwells = 90
wells = list(GFP.columns)[:Nwells]
strains = ["NAO94", "NAO97", "NAO98", "NAO101", "NAO103"]
conditions = ["0uM", "2.5uM", "7.5uM", "12.5uM", "25uM"]
bio_rep = ["A", "B"]
tech_rep = ["a", "b"]
sind = 0
well_dict = {}

for i, well in enumerate(wells):
    entry = {}
    entry["condition"] = conditions[int(np.floor(i / 18))]
    if i % 18 in [0, 1]:
        entry["strain"] = strains[0]
        entry["bio_rep"] = "A"

        if i % 18 == 0:
            entry["tech_rep"] = "a"
        else:
            entry["tech_rep"] = "b"
    else:
        #         print(i)
        ind = i % 18 - 2
        entry["strain"] = strains[int(np.floor(ind / 4)) + 1]
        if ind % 4 == 0:
            entry["bio_rep"] = "A"
            entry["tech_rep"] = "a"
        elif ind % 4 == 1:
            entry["bio_rep"] = "A"
            entry["tech_rep"] = "b"
        elif ind % 4 == 2:
            entry["bio_rep"] = "B"
            entry["tech_rep"] = "a"
        elif ind % 4 == 3:
            entry["bio_rep"] = "B"
            entry["tech_rep"] = "b"
    well_dict[well] = entry
#     print(entry)


df_ind = pd.DataFrame(well_dict).transpose()

In [ ]:
Nwells = 96
wells = list(GFP.columns)[:Nwells]
strains = [
    "NAO94",
    "NAO104",
    "NAO105",
    "NAO106",
    "NAO107",
    "NAO108",
    "NAO109",
    "NAO110",
]
conditions = ["0-0", "0-12.5", "50-12.5", "0-250"]
tech_rep = ["a", "b", "c"]
sind = 0
well_dict = {}

for i, well in enumerate(wells):
    entry = {}
    entry["condition"] = conditions[int(np.floor(i / 24))]
    #         print(i)
    ind = i % 24
    entry["strain"] = strains[int(np.floor(ind / 3))]
    entry["tech_rep"] = tech_rep[ind % 3]
    well_dict[well] = entry
#     print(entry)


df_ind = pd.DataFrame(well_dict).transpose()

In [ ]:
fig, axes = plt.subplots(nrows=8, ncols=2)

for i, strain in enumerate(strains):
    wids = idx[
        df_ind[(df_ind.strain == strain) & (df_ind.tech_rep == "a")]
    ].index.tolist()
    df.loc[:, idx[wids, "GFP"]].plot(
        logy=True, figsize=(20, 30), ax=axes[i, 0], title=strain
    )
    df.loc[:, idx[wids, "OD"]].plot(
        logy=True, figsize=(20, 30), ax=axes[i, 1], title=strain
    )

# for i,cond in enumerate(conditions):
#     wids = idx[df_ind[(df_ind.condition==cond)]].index.tolist()
#     df.loc[:, idx[wids,'GFP']].plot(logy=True,figsize=(20,30),ax=axes[i,0],title=cond)
#     df.loc[:, idx[wids,'OD']].plot(logy=True,figsize=(20,30),ax=axes[i,1],title=cond)

In [ ]:
fig, axes = plt.subplots(nrows=8, ncols=2)

# for i,cond in enumerate(conditions):
#     wids = idx[df_ind[(df_ind.condition==cond)]].index.tolist()
#     df.loc[:, idx[wids,'GFP']].plot(logy=True,figsize=(25,40),ax=axes[i,0],title=cond,ylim=[5e2,1e6],cmap='tab20')
#     df.loc[:, idx[wids,'OD']].plot(logy=True,figsize=(25,40),ax=axes[i,1],title=cond,cmap='tab20')

for i, strain in enumerate(strains):
    wids = idx[df_ind[(df_ind.strain == strain)]].index.tolist()
    df.loc[:, idx[wids, "GFP"]].plot(
        logy=True,
        figsize=(25, 40),
        ax=axes[i, 0],
        title=strain,
        ylim=[5e2, 5e6],
        cmap="tab20",
    )
    df.loc[:, idx[wids, "OD"]].plot(
        logy=False, figsize=(25, 40), ax=axes[i, 1], title=strain, cmap="tab20"
    )

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=1)
for i, cond in enumerate(conditions):
    # for i,strain in enumerate(strains):
    #     wids = idx[df_ind[(df_ind.strain==strain) & (df_ind.bio_rep=='A')]].index.tolist()
    wids = idx[df_ind[(df_ind.condition == cond)]].index.tolist()

    dfOD = df.loc[:, idx[wids, "OD"]]
    dfOD.columns = dfOD.columns.droplevel(level=1)
    dfGFP = df.loc[:, idx[wids, "GFP"]]
    dfGFP.columns = dfGFP.columns.droplevel(level=1)
    (dfGFP / dfOD).plot(
        logy=True, figsize=(10, 40), ax=axes[i], title=cond, cmap="tab20"
    )

In [ ]:
idx = pd.IndexSlice
df2.loc[:, idx[["A1", "A2"], "GFP"]] / df2.loc[:, idx[["A1", "A2"], "OD"]]

In [ ]:
ratio = df2[["A1", "A2"]]["GFP"].div(df2[["A1", "A2"]]["OD"])

In [ ]:
ratio.plot()